## Configuring Your Foundation Model with the OpenAI Python Library

This section outlines how to configure your foundation model using the OpenAI Python library. This library provides a standardized and convenient way to interact with OpenAI's REST API from Python 3.8+ applications.

#### Leveraging the OpenAI Python Library ####

Instead of relying on a custom setup, we recommend using the official OpenAI Python library for interacting with OpenAI models. This library offers:

- Type Definitions: Includes type definitions for all request parameters and response fields.
- Synchronous and Asynchronous Clients: Provides both synchronous and asynchronous clients powered by httpx.
- Standardized Interface: Offers a consistent and well-documented interface for interacting with various OpenAI models.

#### Installation ####

Install the library using pip:

pip install openai

#### Usage: Chat Completions API ####

The Chat Completions API is the recommended way to generate text from the model. Here's a basic example:

```
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4-turbo",  # Replace with your desired model
    messages=[
        {
            "role": "user",
            "content": "Your prompt here.",
        },
    ],
)

print(completion.choices[0].message.content)
```


#### Key Components for Evaluation ####

For the evaluation framework to function correctly, the completion object returned by the client.chat.completions.create() method needs to provide the following information:

- completion.choices[0].message.content: This attribute contains the text generated by the model. The evaluation framework expects this to be the primary output.
- completion.usage: This attribute contains usage statistics. The evaluation framework may use the total_tokens, completion_tokens, and prompt_tokens keys within completion.usage for limiting or tracking resource consumption during evaluation.

#### Configuration Notes ####

- Authentication: The OpenAI Python library handles authentication using API keys. Ensure you have set your API key as an environment variable (OPENAI_API_KEY) or configure it directly when initializing the OpenAI client. Refer to the OpenAI documentation for details on authentication.
- Model Selection: The model parameter in the client.chat.completions.create() method specifies the model to use. Choose the appropriate model for your evaluation task.
- Customization: The client.chat.completions.create() method accepts various parameters for customizing the model's behavior, such as temperature, top_p, and frequency_penalty. Refer to the OpenAI API documentation for a complete list of available parameters.


## Get data

Here we load a dummy data that is presented as a pandas dataframe:

In [2]:
import pandas as pd

notes = {"notes": {
        "1": "Patient ID: Bertha James is a 78-year-old female.\n\nShe came into the office complaining of vaginal bleeding. Additional concerns were raised due to her recent, seemingly unexplainable weight loss.\n\n\nMs. Bertha is otherwise normally healthy for her age but has been demonstrating symptoms of nutritional deficiency. \n\n\nUpon examination, vaginal bleeding is present. The patient appears frail, her weight is less than her average, raising concerns regarding her nutrition and well-being.\n\n\nThe main concern is her vaginal bleeding and unusual weight loss in addition to her overall physical state. We will start by ordering a laboratory workup to better understand the source of the post-menopausal vaginal bleeding including hormonal profile, INR study, and endometrial biopsy. A dietician consultation will be made to assess her nutritional status as her weight loss is concerning. \n\nFurthermore, to ensure her comprehensive care, she has been advised to check with a Psychiatrist for her reported acute insomnia and anxiety, an Orthopedic practitioner for her new-onset hip fracture, and a Dermatologist for proper treatment of her untreated skin fungal infection. Regardless, given the suspicious circumstances surrounding her rapidly changing health status, a report for potential elder abuse will be filed and legal consultation will be sought to ensure the patient's needs and rights are being maintained. \n\n",
        "2": "Clinical Note:\n\nAssessment of Bertha James, 78-year-old female patient, continues to reveal alarming signs that necessitate further attention and evaluation. Primary concerns originated from her initial consultation for vaginal bleeding and subsequent uncovering of atypical weight loss. Soon after, Bertha revealed signs of insomnia and anxiety putting her mental health in question. Instances of physical harm, notably a hip fracture and a fungal infection which remained untreated, are highly suggestive of elder abuse. Previous assessments indicate a suspected lack of proper care adding fuel to these speculations. Legal intervention has been proposed following these observed signs in relation to possible senior abuse. It is crucial that we put in place enhanced measures to ensure Bertha's welfare and safety. Closure and resolution to these troubling indications are paramount to her overall wellbeing. Her caregivers, medical and mental health, need to provide extensive and multidimensional care moving forward.",
 }}

input_df = pd.DataFrame(notes)
input_df.index = [i for i in range(input_df.shape[0])]
input_df.index.name = "noteid"

input_df

/Users/bvg228/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,notes
noteid,
0,Patient ID: Bertha James is a 78-year-old fema...
1,"Clinical Note:\n\nAssessment of Bertha James, ..."


## Setting up the instrument and Running the Evaluations

The run_pipeline function provides a convenient way to evaluate a dataset using different pre-defined prompt categories. It encapsulates the entire evaluation process, handling prompt creation, model completion, and result aggregation behind the scenes.

Here's how it works:

**1. Input Requirements:** The run_pipeline function requires two arguments:

- `input_df`: A pandas DataFrame containing the data to be evaluated.  Each row of the DataFrame represents a single data point to be processed.
- `completion`: The model completion function as indicated above. This function takes a model and a prompt (a list of messages) as input and returns the model's raw output.  This function is responsible for interacting with the language model.

<br>

**2. Prompt Selection:** The function iterates through a dictionary of prompt categories (e.g., "complete", "clinical_reasoning"). Each category is associated with a specific prompt creation function (e.g., create_complete_prompt). These prompt creation functions are designed to be compatible with the input data format. Selection of a pre-formed prompt is done via these prompt creation functions.

<br>

**3. Evaluation Initialization:** For each prompt category, an Evaluation instance is created. This instance is initialized with:

   - completion_fn: The model completion function.
   - prep_fn: The prompt creation function for the current category. This function takes a namedtuple representing a single row from the input DataFrame (obtained via pandas.DataFrame.itertuples) and transforms it into a messages array suitable for the completion_fn.
   - log_enabled: A flag to enable logging of raw model outputs. If True, raw outputs are saved to evaluation/logs/raw_content_&lt;TIMESTAMP&gt;.jsonl.
   - max_tokens: An optional token limit to abort the evaluation loop if exceeded. Default: 80_000
    
<br>

**4. Dataset Evaluation:** The evaluator.run_dataset function is called with the input DataFrame. This function performs the core evaluation loop, processing the DataFrame row by row. Behind the scenes, run_dataset performs the following steps for each row:

   - prompt = prep_fn(namedtuple[dataframe itertuples]): The prompt is generated using the prep_fn (prompt creation function) and a namedtuple representing the current row of the DataFrame.
   - raw_output = completion_fn(model, prompt): The generated prompt is passed to the completion_fn (model completion function) to obtain the model's raw output.
   - response, usage = post_process_fn(raw_output): The raw output is processed by a post_process_fn (defaulting to extracting a single completion and attempting to parse it as JSON). This function extracts the relevant information from the model's response (e.g., the completion text) and returns it along with token usage information.


<br>

**5. Result Aggregation:** The run_dataset function returns a dictionary where keys are 'noteid's (presumably from the input DataFrame) and values are the parsed completion results (or whatever the first output of the post_process_fn returns). The run_pipeline function aggregates these results across all prompt categories. For each note, it creates a dictionary containing the evaluation grades for each category. Thus, each note is graded for each of the categories.

<br>

**6. Output:** The run_pipeline function returns a dictionary where keys are 'noteid's and values are dictionaries containing the evaluation grades for each category (e.g., {&#039;noteid1&#039;: {&#039;complete&#039;: 1, &#039;clinical_assessment_reasoning&#039;: 0, ...}}).

### Example usage

In [3]:
from run_5cs_pipeline import run_pipeline
graded_dict = run_pipeline(input_df, completion)


--- Running evaluation for: complete ---

--- Running evaluation for: clinical_assessment_reasoning ---

--- Running evaluation for: contingent ---

--- Running evaluation for: concise ---

--- Running evaluation for: correct ---


In [4]:
graded_dict

{0: {'complete': 1,
  'clinical_assessment_reasoning': 1,
  'contingent': 0,
  'concise': 1,
  'correct': 1},
 1: {'complete': 0,
  'clinical_assessment_reasoning': 1,
  'contingent': 0,
  'concise': 1,
  'correct': 0}}

In [5]:
graded_df = pd.DataFrame.from_dict(graded_dict, orient='index')
graded_df

,complete,clinical_assessment_reasoning,contingent,concise,correct
0,1,1,0,1,1
1,0,1,0,1,0
